In [25]:
import requests
import json
from scrapy.http import HtmlResponse
import os
import datetime
import boto3
import copy
import hashlib

url = f"https://rewardsforjustice.net/index/?jsf=jet-engine:rewards-grid&tax=cyber:857&pagenum=1&nocache=1650868733"
# url = f"https://rewardsforjustice.net/index/?jsf=jet-engine%3Arewards-grid&tax=crime-category%3A1073&pagenum=1&nocache=1650868733"

payload = "action=jet_engine_ajax&handler=get_listing&page_settings%5Bpost_id%5D=22076&page_settings%5Bqueried_id%5D=22076%7CWP_Post&page_settings%5Belement_id%5D=ddd7ae9&page_settings%5Bpage%5D=1&listing_type=elementor&isEditMode=false&addedPostCSS%5B%5D=22078"
headers = {
    'authority': 'rewardsforjustice.net',
    'method': 'POST',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,hi;q=0.6,gu;q=0.5',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest'
}

res = requests.post(url, headers=headers, data=payload)
body = json.loads(res.text)
data = body['data']['html']

In [26]:
ress = HtmlResponse(url="example.com",body=data,encoding='utf-8')
                
body = ress.xpath('//*[@class="jet-engine-listing-overlay-wrap"]//@data-url').extract()
body

['https://rewardsforjustice.net/rewards/foreign-malicious-cyber-activity-against-u-s-critical-infrastructure/',
 'https://rewardsforjustice.net/rewards/marat-valeryevich-tyukov/',
 'https://rewardsforjustice.net/rewards/mikhail-mikhailovich-gavrilov/',
 'https://rewardsforjustice.net/rewards/pavel-aleksandrovich-akulov/',
 'https://rewardsforjustice.net/rewards/evgeny-viktorovich-gladkikh/']

In [20]:
d = ress.xpath('//*[@class="jet-engine-listing-overlay-wrap"]')
d[0].xpath(".//@data-url").get()


'https://rewardsforjustice.net/rewards/ali-al-shair/'

In [12]:
import requests
import json
from scrapy.http import HtmlResponse
import os
import datetime
import boto3
import copy
import hashlib
class Data():
    try:
        counter = 0
        final_list = []
        date = datetime.date.today()
        yesterday = date - datetime.timedelta(days=1)
        inputpath = f'InputFiles'
        outputpath = f'OutputFiles'
        Differencepath = f'DifferenceFiles'
        os.mkdir(inputpath)
        os.mkdir(outputpath)
        os.mkdir(Differencepath)
    except:
        pass

    def uploadToS3(self):
        try:
            print("uploading files to s3")
            s3 = boto3.client('s3')
            # s3.upload_file(f'{self.inputpath}/{self.date}_input_cbr.ru.xls', "sams-scrapping-data",
            #                f"USA/Original/{self.date}_input_cbr.ru.xls")
            s3.upload_file(f'{self.outputpath}/{self.date}_output_rewardsforjustice.net_Entity.json', "sams-scrapping-data",
                           f"USA/Parced/{self.date}_output_rewardsforjustice.net_Entity.json")
            s3.upload_file(f'{self.Differencepath}/{self.date}_DifferenceFile_rewardsforjustice.net_Entity.json', "sams-scrapping-data",
                           f"USA/Diffrance/{self.date}_DifferenceFile_rewardsforjustice.net_Entity.json")
            print("uploaded files to s3")
        except Exception as e:
            print(e)

    def compare(self):
        try:
            with open(f'{self.outputpath}/{self.yesterday}_output_rewardsforjustice.net_Entity.json', 'rb') as f:
                data = f.read()
        except:
            # print("---------------------Alert--------------------------")
            print(f"There is not any old file for date: {self.yesterday.ctime()}")
            # print("----------------------------------------------------")
            data = "No DATA"
        if data != "No DATA":
            old_list = json.loads(data)
            new_list = copy.deepcopy(self.final_list)

            new_profiles = []
            removed_profiles = []
            updated_profiles = []

            old_dict = {}
            for val in old_list:
                old_dict[val["uid"]] = val

            new_uid_list = []
            for val1 in new_list:
                new_uid = val1["uid"]
                new_uid_list.append(new_uid)
                if new_uid in old_dict.keys():
                    # print("Already in List")
                    for i in val1:
                        if i != "last_updated":
                            try:
                                if val1[i] != old_dict[val1["uid"]][i]:
                                    print(f"Updataion Detected on: {val1['uid']} for: {i}")
                                    updated_profiles.append(val1)
                                    break
                            except:
                                print(f"Updataion Detected on: {val1['uid']} for: {i}")
                                updated_profiles.append(val1)
                                break


                else:
                    new_profiles.append(val1)
                    print(f"New Profile Detected : {val1['uid']}")

            for val2 in old_dict.keys():
                if val2 not in new_uid_list:
                    print(f"Removed Profile Detected : {val2}")
                    removed_profiles.append(old_dict[val2])

            with open(f'{self.outputpath}/{self.date}_Difference_output_rewardsforjustice.net_Entity.json',
                      "w") as outfile:
                json.dump(new_profiles + updated_profiles, outfile)

            with open(f'{self.date}_RemovedData_File_rewardsforjustice.net_Entity.json', "w") as outfile:
                json.dump(removed_profiles, outfile)

            # with open(f'ofac-sdn-logfile.csv', "a") as outfile:
            #     passing = f"{today_date.ctime()},{input_filename},{output_filename},{total_profile_available},{len(new_list)},{len(new_profiles)},{len(updated_profiles)},{len(new_profiles) + len(updated_profiles)},{len(removed_profiles)},{diffrance_filename},{removed_filename}\n"
            #     outfile.write(passing)

    def parsing(self):
        try:
            for p in range(1,7):
                url = f"https://rewardsforjustice.net/index/?jsf=jet-engine%3Arewards-grid&tax=crime-category%3A1073&pagenum={p}&nocache=1650868733"

                payload = "action=jet_engine_ajax&handler=get_listing&page_settings%5Bpost_id%5D=22076&page_settings%5Bqueried_id%5D=22076%7CWP_Post&page_settings%5Belement_id%5D=ddd7ae9&page_settings%5Bpage%5D=1&listing_type=elementor&isEditMode=false&addedPostCSS%5B%5D=22078"
                headers = {
                    'authority': 'rewardsforjustice.net',
                    'method': 'POST',
                    'accept': 'application/json, text/javascript, */*; q=0.01',
                    'accept-encoding': 'gzip, deflate, br',
                    'accept-language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,hi;q=0.6,gu;q=0.5',
                    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
                    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
                    'x-requested-with': 'XMLHttpRequest'
                }

                res = requests.post(url, headers=headers, data=payload)
                body = json.loads(res.text)
                data = body['data']['html']

                # url = "https://rewardsforjustice.net/index/?jsf=jet-engine:rewards-grid&tax=crime-category:1073&pagenum=1"
                #
                # payload = {}
                # headers = {
                #     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                #     'accept-encoding': 'gzip, deflate, br',
                #     'accept-language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,hi;q=0.6,gu;q=0.5',
                #     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
                # }
                # res = requests.get(url, headers=headers, data=payload)
                ress = HtmlResponse(url="example.com",body=data,encoding='utf-8')
                try:
                    body = ress.xpath('//*[@class="jet-engine-listing-overlay-wrap"]//@data-url').extract()
                    for i in body:
                        data = requests.get(i,headers=headers, data=payload)
                        response = HtmlResponse(url="example.com",body=data.content)
                        try:
                            name = response.xpath('//*[@class="elementor-column elementor-col-33 elementor-top-column elementor-element elementor-element-7a45ba0"]//h2[@class="elementor-heading-title elementor-size-default"]//text()').extract_first(default="").strip()
                        except:
                            name = ""
                        try:
                            comment = ''.join(response.xpath('//*[@class="elementor-element elementor-element-52b1d20 elementor-widget elementor-widget-theme-post-content"]//p//text()').extract()).strip()
                        except:
                            comment = ""
                        item = {}
                        print(name)
                        item['uid'] = hashlib.sha256(((name).lower()).encode()).hexdigest()
                        item['name'] = name
                        item['alias_name'] = []
                        item['country'] = []
                        item['list_type'] = "Entity"
                        item['last_updated'] = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
                        item['entity_details'] = {}
                        item['nns_status'] = False
                        item['address'] = []
                        add = {}
                        add['country'] = ""
                        add['complete_address'] = ""
                        item['address'].append(add)
                        item['documents'] = {}
                        item['comment'] = comment
                        item['sanction_list'] = {}
                        item['sanction_list']['sl_authority'] = "Rewards for Justice, USA"
                        item['sanction_list']['sl_url'] = "https://rewardsforjustice.net/index/?jsf=jet-engine:rewards-grid&tax=crime-category:1073"
                        item['sanction_list']['sl_host_country'] = "USA"
                        item['sanction_list']['sl_type'] = "Sanctions"
                        item['sanction_list']['watch_list'] = "North America Watchlists"
                        item['sanction_list']['sl_source'] = "Rewards for Justice-Wanted for Terrorism List, Russia"
                        item['sanction_list']['sl_description'] = "List of most wanted individuals and entities by Rewards for Justice, USA."
                        item['list_id'] = "USA_T30098"
                        self.final_list.append(item)
                        counter = self.counter + 1
                except Exception as e:
                    print(e)
        except Exception as e:
            print(e)
        try:
            ab = json.dumps(self.final_list)
            print("--->",len(self.final_list))
            f = open(f"{self.outputpath}/{self.date}_output_rewardsforjustice.net_Entity.json", "a")
            f.write(ab)
            f.close()
        except Exception as e:
            print(e)

temp = Data()
temp.parsing()
# temp.compare()
# temp.uploadToS3()

Islamic Revolutionary Guard Corps (IRGC)
Saudi Hizballah
Tehrik-e Taliban Pakistan (TTP)
Revolutionary Struggle (RS)
Revolutionary People’s Liberation Party/Front (DHKP/C)
Revolutionary Organization 17 November (17N)
Movement for Unity and Jihad in West Africa (MUJWA)
Moro National Liberation Front (MNLF)
Lashkar-e Tayyiba (LeT)
Kurdistan Workers Party (PKK)
Jama’at Nusrat al-Islam wal-Muslimin (JNIM)
Islamic State’s Khorasan Province (ISIS-K)
Islamic State West Africa Province (ISIS-WA)
Islamic State in Iraq and Syria (ISIS)
Islamic Revolutionary Guard Corps-Qods Force (IRGC-QF)
Hurras al-Din
Jemaah Islamiyah (JI)
Japanese Red Army (JRA)
ISIS in the Greater Sahara (ISIS-GS)
Hizballah
Hayat Tahrir al-Sham (HTS)
Harakat al-Muqawama al-Islamiya (HAMAS)
Haqqani Network (HQN)
Egyptian Islamic Jihad (EIJ)
Ansar al-Tawhid (Supporters of Monotheism)
Ansar al-Dine (AAD)
Al-Shabaab
Al-Qa’ida in the Lands of the Two Niles (AQTN)
Al-Qa’ida in the Islamic Maghreb (AQIM)
Al-Qa’ida in the Arabian Pe